## 1. Introduction to Diffusion Models

* Forward to Backward Process

`-` Forward Diffusion Process

* 이미지에 가우시안 노이즈를 더함
* 여러 스탭에 걸쳐 가우시안 프로세스로부터 나온 이미지가 됨
* 이미지가 주어졌을 때 얼마나 깨끗하게 만들 수 있는지를 학습
* 초창기 Diffusion의 경우 1 epoc 진행 시 노이즈를 일부 같이 더함 -> 깨끗한 이미지 생성

> 처음에는 물체의 윤곽, 갈수록 디테일 추가
>
> Reverse Diffusion Process의 경우 반대로

`-` Probability Flow ODE

> equation을 바꿈 -> 노이즈가 없어서 수렴이 빠름
>
> 특정 노이즈가 특정 이미지로 매핑됨
>
> 기존 방식에 비해 엄청나게 경량화됨, 일단은

## 2. Faster Solver

### A. Denoising Diffusion Implicit Models DDIM

* 기존 Diffusion은 마코브 체인을 따랐음
* 마코브 체인의 가정을 깸 -> 이미지 + 노이즈에서 노이즈를 완전히 제거하는 방향, Stochastic Term 없음, 더 좋은 샘플러를 고를 수록 빨라질 가능성이 높음

### B. Distillation

* Latent Space -> ㅇㅅㅇ

### C. Consistent Model

* 모델의 space 자체를 바꿈 -> 샘플링을 적게 해도 빠르게 이동 가능
* 코스트가 현실적으로 변함 : 실제 서비스가 폭발적으로...

### D. Diffusion with Data Reuse

* Distillation / Consistent Model의 경우 Diversity가 좀 떨어질 수 있음. GAN에서 나온 현상과 유사
* FRDiff
> 피쳐를 중간에 건너뛴다... 


* 이미지를 중간에 건너뛰는 Reduced NFE -> 디테일이 모자람
* Feature Reuss -> 색이 다 빠짐 ㅇㅅㅇ
> 둘을 어떻게 잘 섞어서 선택적으로 다른 경량화 기법을 적용 -> 경량화 가능

## 3. AR Image Generative Model

* LLM like image generate

> 토큰이 Language가 아닌 image, pixel에 적용


* LlamaGen : 라마 아키텍쳐를 그대로 사용해서 이미지를 생성했는데 잘 됨
> 아무래도 오래 걸림

* Lumina-mGPT : Language로 묘사하고 다양한 사진을 한번에 학습했더니 잘 되더라

상호 장점을 이용하려고 연관짓는 중임 ㅇㅇ

## 4. Acceleration

* AR Gen -> 굳이 순차적으로 만들어야 하나? 다른 오더대로 만들면 안되나?
> 무조건 샘플을 뽑고 다음 샘플을 뽑아야만 했는데, 여러 개를 한번에 뽑을 수 있게 됨
>
> AR을 깨서 가속

## 5. Speculative Decoding

* 큰 모델 하나로 만드려 하니, 너무 느림. 샘플링 하나하나...
> 작은 모델을 가져오고 생성하라고 함 -> 병렬적으로 큰 모델이 검사. 몇 개가 맞았나?
>
> 많은 경우가 맞긴 함, 틀린 애는 반려하고 다시 만들라고 함. 반복
>
> 작은 모델이 drafting, 큰 모델이 검사. 큰 모델이 좋아야 함.

* 이미지의 경우 작은 모델을 만들기 어려움

* Jacobi Decoding
> fixed point : $f(x_0) = x_0$인 $x_0$
> 
> 함수에 입력을 넣어 출력을 뽑는데, 다른 값이 나온다 -> fixed point에 원래 넣던 입력보단 가까움 -> 계속 이동... Jacobi Decoding. LLM도 비슷하다고 가정
>
> 한번에 만들라고 함 -> 또 만들라고 함. 좀 맞음 -> 계속. 근데 생각보다 속도 향상 효과가 그렇게 안나옴... ㅇㅅㅇ 근데 Visual Model에 넣었더니, 성능 많이 향상
>
> AR gen의 경우 토큰이 중간에 많이 틀려도, 생성된 이미지 퀼은 그렇게 차이 안남 -> 많이 맞음. 이미지 퀄이 많이 안 낮아져도 빨라짐

## 6. Limitation of Speculative Decoding

* approximate 기법 사용 시, 이미지 퀄은 괜찮지만 확률 분포가 차이가 생각보다 많이 남
> Visually Well Token이 많으면 받아들이고, 아니면 Reject하는 식으로 하면... 
>
> Original Distribution을 거의 따라가면서 생성할 수 있음. 스피드업과 이미지 생성의 안정성 둘 다 챙김